In [1]:
import pandas as pd
import math
import re

In [2]:
re_han = re.compile("([\u4E00-\u9FD5]+)", re.U)

def txtcnt(txt):
    cnt = 0
    blocks = re_han.split(txt)
    for blk in blocks:
        if not blk:
            continue
        if re_han.match(blk):
            cnt += len(blk)
    return cnt

def tokens(txt,word_len):
    words = []
    blocks = re_han.split(txt)
    for blk in blocks:
        if not blk:
            continue
        if re_han.match(blk):
            words += ngram(blk,word_len)
    return words

def ngram(txt,word_len):
    words = []
    tmp_txt = txt
    remain_len = len(tmp_txt) - word_len
    if remain_len <= 0:
        words.append(tmp_txt)
    else:
        inx = 0
        while inx <= remain_len:
            word = tmp_txt[inx:inx+word_len]
            words.append(word)
            inx += 1
    return words

In [3]:
word_len = 6
line_inx = 1
file_name = 'comment_word_'+ str(word_len) +'.txt'
with open('comment_all.txt','r') as f \
    ,open(file_name,'w') as w:
    for line in f.readlines():
        words = tokens(line,word_len)
        for word in words:
            w.write(word + ',' + str(line_inx) +',1\n')
        line_inx += 1

In [4]:
df = pd.read_csv(file_name,header=None)
df[:2]

,0,1,2
0,难受,1,1
1,问题不在福祉,2,1


In [5]:
idf = df.filter(items=[0,1])

In [6]:
len(idf)

37971

In [7]:
idf_df = idf.drop_duplicates()

In [8]:
idf_df1 = idf_df.groupby(by=0).count()

In [9]:
idf_df1[:5]

,1
0,
一,3
一一从中读到,1
一一张罚单丢,1
一万这还不包,1
一下不就行了,1


In [10]:
tf = df.filter(items=[0,2])

In [11]:
tf_df = tf.groupby(by=0).sum()

In [12]:
tf_df[:5]

,2
0,
一,3
一一从中读到,1
一一张罚单丢,1
一万这还不包,1
一下不就行了,1


In [13]:
tf_df.sort_values(by=2,ascending=False).to_csv('词频.csv')

In [14]:
tf_idf = idf_df1.join(tf_df)

In [15]:
idf_max = tf_idf[1].max()
tf_max = tf_idf[2].max()

tf_idf['idf'] = tf_idf[1].apply(lambda x: math.log(idf_max/x))
tf_idf['tf'] = tf_idf[2].apply(lambda x: x/tf_max)

In [16]:
tf_idf['score'] = tf_idf['idf'] * tf_idf['tf']

In [17]:
tf_idf[:5]

,1,2,idf,tf,score
0,,,,,
一,3,3,2.427748,0.088235,0.214213
一一从中读到,1,1,3.526361,0.029412,0.103716
一一张罚单丢,1,1,3.526361,0.029412,0.103716
一万这还不包,1,1,3.526361,0.029412,0.103716
一下不就行了,1,1,3.526361,0.029412,0.103716


In [18]:
tf_idf1 = tf_idf.reset_index()

In [19]:
tf_idf1.sort_values(by='score',ascending=False)

,0,1,2,idf,tf,score
9144,卫星定位装置,6,10,1.734601,0.294118,0.510177
6941,元,14,16,0.887303,0.470588,0.417554
11009,哈哈哈哈哈哈,2,5,2.833213,0.147059,0.416649
27846,罚款,7,8,1.580450,0.235294,0.371871
18623,支持,7,8,1.580450,0.235294,0.371871
...,...,...,...,...,...,...
12073,基本利益得到,1,1,3.526361,0.029412,0.103716
12072,基本不花钱,1,1,3.526361,0.029412,0.103716
12071,基本上路的原,1,1,3.526361,0.029412,0.103716
35413,龙芯要和,1,1,3.526361,0.029412,0.103716


In [20]:
tf_idf1[(tf_idf1[2]>=3) & (tf_idf1[1]>=3)].sort_values(by='score',ascending=False)

,0,1,2,idf,tf,score
9144,卫星定位装置,6,10,1.734601,0.294118,0.510177
6941,元,14,16,0.887303,0.470588,0.417554
27846,罚款,7,8,1.580450,0.235294,0.371871
18623,支持,7,8,1.580450,0.235294,0.371871
28990,莫名掉线不能,12,12,1.041454,0.352941,0.367572
...,...,...,...,...,...,...
15815,开运输行业各,3,3,2.427748,0.088235,0.214213
16308,必将会计入成,3,3,2.427748,0.088235,0.214213
35381,鹭鸶腿上劈精,3,3,2.427748,0.088235,0.214213
11099,唉,30,30,0.125163,0.882353,0.110438


In [21]:
tf_idf1.sort_values(by='score',ascending=False).to_csv('score.csv',index=False)

In [22]:
tf_idf1[:1]

,0,1,2,idf,tf,score
0,一,3,3,2.427748,0.088235,0.214213


In [23]:
import math

math.log(10.0)

2.302585092994046